# Installing *torch-summary*

In [ ]:
!pip install torch-summary

# Importing necessary libraries

In [ ]:
import scipy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
from torchsummary import summary
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Checking if GPU is available

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("GPU Runtime Detected")

else:
  device = torch.device("cpu")
  print("No GPU Found - CPU Runtime")

# Defining the *splitter* function

In [ ]:
def splitter(array, win_len, hop_len, return_df = True):
   N = array.shape[0]
   m = 0
   ids = []
   while m + win_len < N:
      ids.append([m, m + win_len])
      m = m + hop_len
      
   if return_df:
      return pd.DataFrame([array[i[0]: i[1]] for i in ids])
   else:
      return np.array([array[i[0]: i[1]] for i in ids])

# Loading the Dataset

In [ ]:
file = "../input/engine-acoustic-emissions/dataset.mat"
dataset = scipy.io.loadmat(file)
dataset.keys()

# Preprocessing

## Splitting the original Dataset

In [ ]:
win_len, hop_len = 8000, 1000
df_list = []

for state in ['normal', 'inner', 'roller', 'outer']:
    temp_df = splitter(dataset[state].reshape((-1,)), win_len, hop_len, return_df = True)
    temp_df['state'] = state
    df_list.append(temp_df)

df = pd.concat(df_list).reset_index(drop = True)
df

## Train/Test Splitting

In [ ]:
df_train, df_test = train_test_split(df, test_size = 0.3, random_state = 42)

## Feature/Target Declaration

In [ ]:
x_train, x_test = df_train.iloc[:, :-1], df_test.iloc[:, :-1]
y_train, y_test = df_train.iloc[:, -1], df_test.iloc[:, -1]

## Defining *STFT* function

In [ ]:
def stft(signals, window_len, hop_len, freq_filter = None, window = None):
  """
  STFT(signals, window_len, hop_len, freq_filter = None, window = None) - Application of Short-Time Fourier Transform to derive Time-Frequency representation of the inputted signals

  Arguemnts:
  signals -- A pd.DataFrame() incuding signals in its rows.
  window_len -- Lenght of the desired time segments.
  hop_len -- Length of the feed, used to get forward during the segmentation process.
  freq_filter -- A frequency filter object from scipy.signal module (e.g. scipy.signal.butter()) to avoid aliasing.
  window -- A window object from scipy.signal.windows module (e.g. scipy.signal.windows.hann()) to encounter the leakage error.

  Return Value:
  A np.array(), whose first dimension equals the number of rows included in the input pd.DataFrame; it includes derived Time-Frequency representations of the inputted signals. freq_filter and window are not mandatory 
  arguments and a function call without them is valid, however, we recommend using them to avoid aliasing (and of course near-zero/DC filtering through band-pass filters) and leakage error. Pay attention that unlike
  the case of damavand.signal_processing.fft() or damavand.signal_processing.zoomed_fft(), for this function you have to define freq_filter and window objects with a lenght that suits the segmented signals (equal to
  the window_len argument), instead of the original signals, presented in the inputted pd.DataFrame().

  Descriptions:
  By the application of this function, one is able to derive Time-Frequency representation; this is done by first segmenting the original signals to a series of shorter signals and consecutively FFT is applied on each
  segmented signal to derive the corresponding frequency representation. Results are usually visualized as heatmaps, whose vertical axis is the frequency dimension and the horizontal one is left to time dimension. One
  can use the damavand.utils.STFT_axises() to generate both time and frequency axises needed to visualize the resulting heatmaps.
  """
  splitted_signals = np.array(signals.apply(splitter, args = (window_len, hop_len, False), axis = 1).to_list())
  if freq_filter is not None:
    splitted_signals = scipy.signal.sosfilt(freq_filter, splitted_signals)

  if window is not None:
    splitted_signals = splitted_signals * window

  return 2.0/splitted_signals.shape[2] * np.abs(scipy.fft.fft(splitted_signals)[:, :, 0:splitted_signals.shape[2]//2])

## STFT

In [ ]:
STFT_window = scipy.signal.windows.hann(1000)
STFT_freq_filter = scipy.signal.butter(25, [5, 4500], 'bandpass', fs = 10000, output='sos')
x_train_STFT = stft(x_train, 1000, 100, STFT_freq_filter, STFT_window)
x_test_STFT = stft(x_test, 1000, 100, STFT_freq_filter, STFT_window)
print(x_train_STFT.shape, x_test_STFT.shape)

## Feature Scaling

In [ ]:
x_train_STFT_scaled = []
for i in range(x_train_STFT.shape[0]):
    x_train_STFT_scaled.append(x_train_STFT[i, :, :] / np.max(x_train_STFT[i, :, :]))
    
x_train_STFT_scaled = np.array(x_train_STFT_scaled)
x_train_STFT_scaled.shape

In [ ]:
x_test_STFT_scaled = []
for i in range(x_test_STFT.shape[0]):
    x_test_STFT_scaled.append(x_test_STFT[i, :, :] / np.max(x_test_STFT[i, :, :]))
    
x_test_STFT_scaled = np.array(x_test_STFT_scaled)
x_test_STFT_scaled.shape

## Target Encoding

In [ ]:
labelencoder= LabelEncoder()
y_train_encoded = labelencoder.fit_transform(y_train.to_numpy())
y_test_encoded = labelencoder.transform(y_test.to_numpy())

# Training

## Model Declaration

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.conv = torch.nn.Conv2d(in_channels = 1, out_channels = 5, kernel_size = 5)
        self.avgPool = torch.nn.AvgPool2d(kernel_size = 5)
        self.fc = torch.nn.Linear(5 * 13 * 99, 4)

    def forward(self, x):

        z = self.conv(x)
        z = torch.tanh(z)
        z = self.avgPool(z)

        z = z.view(-1, 5 * 13 * 99)

        z = self.fc(z)

        return z

In [ ]:
model = Classifier().to(device)
summary(model, (1, 70, 500));

## Fitting the Model

In [ ]:
x_train_train, x_train_validation, y_train_train, y_train_validation = train_test_split(x_train_STFT_scaled, y_train_encoded, test_size = 0.25)

x_train_train = x_train_train.reshape(-1, 1, 70, 500)
x_train_validation = x_train_validation.reshape(-1, 1, 70, 500)

x_train_VAR = torch.autograd.Variable(torch.Tensor(x_train_train).float()).to(device)
y_train_VAR = torch.autograd.Variable(torch.LongTensor(y_train_train)).to(device)
x_valid_VAR = torch.autograd.Variable(torch.Tensor(x_train_validation).float()).to(device)
y_valid_VAR = torch.autograd.Variable(torch.LongTensor(y_train_validation)).to(device)

In [ ]:
lr = 0.005
ep = 50

model = Classifier().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = lr,
                             weight_decay = lr / ep)

losses = []
valid_losses = []
accs = []
valid_accs = []



for epoch in range(ep):

  # validation step
  valid_loss = criterion(model(x_valid_VAR), y_valid_VAR).item()
  valid_losses.append(valid_loss)
  valid_acc = accuracy_score(y_train_validation, np.argmax(model(x_valid_VAR).cpu().detach().numpy(), axis = 1))
  valid_accs.append(valid_acc)

  # training step
  optimizer.zero_grad()
  loss = criterion(model(x_train_VAR), y_train_VAR)
  acc = accuracy_score(y_train_train, np.argmax(model(x_train_VAR).cpu().detach().numpy(), axis = 1))
  accs.append(acc)
  losses.append(loss.item())
  loss.backward()
  optimizer.step()
  print(f"Epoch {epoch+1}, loss: {np.round(loss.item(), 4)}  , Vloss: {np.round(valid_loss, 4)}, acc: {np.round(acc, 4)}, Vacc: {np.round(valid_acc, 4)}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,4))
fig.suptitle('Deep Learning Model Training Process')
axes[0].plot(losses, label='Training Loss')
axes[0].plot(valid_losses, label='Validation Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Vs. Epochs')
axes[0].legend()

axes[1].plot(accs, label='Training Accuracy')
axes[1].plot(valid_accs, label='Validation Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy Vs. Epochs')
axes[1].legend()
plt.show()

# Evaluation

## Test-set Accuracy

In [ ]:
x_test_VAR = torch.autograd.Variable(torch.Tensor(x_test_STFT_scaled.reshape(-1, 1, 70, 500)).float()).to(device)
testing_acc = accuracy_score(y_test_encoded, np.argmax(F.softmax(model(x_test_VAR), dim = 0).cpu().detach().numpy(), axis = 1))
print('Held-out Test-set Accuracy: ', round(testing_acc, 4))

## Test-set Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_test_pred = np.argmax(F.softmax(model(x_test_VAR), dim = 0).cpu().detach().numpy(), axis = 1)
y_test_pred_decoded = labelencoder.inverse_transform(y_test_pred)

matrix = confusion_matrix(y_test_encoded, y_test_pred)

ax = sns.heatmap(matrix, annot=True, fmt='d', cbar = True, square = True, cmap = 'Blues')
ax.set_xlabel("Predicted", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(np.unique(y_test_pred_decoded))
ax.set_ylabel("Actual", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(np.unique(y_test_pred_decoded))
ax.set_title("Confusion Matrix", fontsize=14, pad=20)
plt.show()